In [1]:
import warnings

warnings.filterwarnings("ignore", message=r".*GIL", category=RuntimeWarning)

In [2]:
import json
import re
import subprocess

import polars as pl

pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)
pl.Config.set_fmt_str_lengths(1000);

In [3]:
RUFF = "./ruff.26230b1ed3"
linter_data = json.loads(
    subprocess.run(
        [RUFF, "linter", "--output-format=json"],
        check=True,
        text=True,
        capture_output=True,
    ).stdout
)
linters = {linter["name"]: linter["prefix"] for linter in linter_data}
rule_data = json.loads(
    subprocess.run(
        [RUFF, "rule", "--all", "--output-format=json"],
        check=True,
        text=True,
        capture_output=True,
    ).stdout
)
rules = {rule["code"]: rule["name"] for rule in rule_data}
rule_to_linter = {rule["code"]: linters[rule["linter"]] for rule in rule_data}

## Rule candidates

In [4]:
full = pl.read_csv("out.csv").with_columns(
    (
        pl.col("accuracy")
        + 2 * pl.col("severity")
        + 0.5 * pl.col("fixability")
        + pl.col("applicability")
        + pl.col("configuration")
        + pl.col("conflicts")
    ).alias("total"),
    pl.col("rule").replace_strict(rule_to_linter).alias("linter"),
    pl.col("rule").replace_strict(rules).alias("name"),
)

In [5]:
full.filter(
    pl.col("total") >= 9,
    pl.col("type") == "Stable",
).height

578

In [6]:
full.filter(
    pl.col("accuracy") == 2,
    pl.col("total") >= 9,
    pl.col("type") == "Stable",
).height

551

In [7]:
# Just requiring a total score >= 9 doesn't really narrow things down enough, even with
# a required accuracy of 2 and limiting to stable rules. Let's impose a few more constraints
# such as no conflicts or redundancy with other tools, a high applicability, and also a high
# severity. This actually requires a total score >= 10 so the >= 9 check is now redundant.
#
# Expanding the applicabilty range to >= 1 just pulls in the pytest rules. pytest was basically
# (maybe literally) the only library I considered widespread enough to warrant a 1.
#
# 197 seems like a pretty reasonable number of default rules (out of 938), just percentage-wise.
df = full.filter(
    pl.col("accuracy") == 2,
    pl.col("type") == "Stable",
    pl.col("conflicts") == 2,
    pl.col("applicability") == 2,
    pl.col("severity") == 2,
)
df.sort("rule")

rule,type,accuracy,severity,fixability,applicability,configuration,conflicts,total,linter,name
str,str,i64,i64,i64,i64,i64,i64,f64,str,str
"""A002""","""Stable""",2,2,0,2,1,2,11.0,"""A""","""builtin-argument-shadowing"""
"""A004""","""Stable""",2,2,0,2,1,2,11.0,"""A""","""builtin-import-shadowing"""
"""A005""","""Stable""",2,2,0,2,1,2,11.0,"""A""","""stdlib-module-shadowing"""
"""A006""","""Stable""",2,2,0,2,1,2,11.0,"""A""","""builtin-lambda-argument-shadowing"""
"""ARG001""","""Stable""",2,2,0,2,1,2,11.0,"""ARG""","""unused-function-argument"""
"""ARG002""","""Stable""",2,2,0,2,1,2,11.0,"""ARG""","""unused-method-argument"""
"""ARG003""","""Stable""",2,2,0,2,1,2,11.0,"""ARG""","""unused-class-method-argument"""
"""ARG004""","""Stable""",2,2,0,2,1,2,11.0,"""ARG""","""unused-static-method-argument"""
"""ARG005""","""Stable""",2,2,0,2,1,2,11.0,"""ARG""","""unused-lambda-argument"""


## Comparison with current defaults

In [8]:
# fmt: off
current_defaults = [
    "E401",	    "E742",	    "F504",	    "F602",	    "F811",
    "E712",	    "F404",	    "F521",	    "F634",	    "F901",
    "E741",	    "F503",	    "F601",	    "F722",	    "E711",
    "F403",	    "F509",	    "F633",	    "F842",	    "E731",
    "F502",	    "F541",	    "F707",	    "E703",	    "F402",
    "F508",	    "F632",	    "F841",	    "E722",	    "F501",
    "F525",	    "F706",	    "E702",	    "F401",	    "F507",
    "F631",	    "F823",	    "E721",	    "F407",	    "F524",
    "F704",	    "E701",	    "E902",	    "F506",	    "F622",
    "F822",	    "E714",	    "F406",	    "F523",	    "F702",
    "E402",	    "E743",	    "F505",	    "F621",	    "F821",
    "E713",	    "F405",	    "F522",	    "F701",
]
# fmt: on
len(current_defaults)

59

In [9]:
in_defaults = pl.col("rule").is_in(current_defaults)
len(df.filter(in_defaults))

42

In [10]:
missing_current_defaults = set(current_defaults) - set(df["rule"])
len(missing_current_defaults)

17

In [11]:
# looks like these all got filtered out for severity == 1.
# I don't have any strong opposition to these and would probably lean toward
# including them to make the new default set a superset of the existing defaults.
full.filter(pl.col("rule").is_in(missing_current_defaults)).sort("rule")

rule,type,accuracy,severity,fixability,applicability,configuration,conflicts,total,linter,name
str,str,i64,i64,i64,i64,i64,i64,f64,str,str
"""E401""","""Stable""",2,1,2,2,2,2,11.0,"""""","""multiple-imports-on-one-line"""
"""E402""","""Stable""",2,1,0,2,2,2,10.0,"""""","""module-import-not-at-top-of-file"""
"""E701""","""Stable""",2,1,0,2,2,1,9.0,"""""","""multiple-statements-on-one-line-colon"""
"""E702""","""Stable""",2,1,0,2,2,1,9.0,"""""","""multiple-statements-on-one-line-semicolon"""
"""E703""","""Stable""",2,1,2,2,2,1,10.0,"""""","""useless-semicolon"""
"""E711""","""Stable""",2,1,2,2,2,2,11.0,"""""","""none-comparison"""
"""E712""","""Stable""",2,1,2,2,2,2,11.0,"""""","""true-false-comparison"""
"""E713""","""Stable""",2,1,2,2,2,2,11.0,"""""","""not-in-test"""
"""E714""","""Stable""",2,1,2,2,2,2,11.0,"""""","""not-is-test"""


In [12]:
# looks like these were filtered out for accuracy = 1, I think I was
# probably too harsh here. There's an open bug report from dscorbett
# with a false negative, which is the only reason I can identify for this
full.filter(pl.col("rule").is_in(("A001", "A003")))

rule,type,accuracy,severity,fixability,applicability,configuration,conflicts,total,linter,name
str,str,i64,i64,i64,i64,i64,i64,f64,str,str
"""A001""","""Stable""",1,2,0,2,1,2,10.0,"""A""","""builtin-variable-shadowing"""
"""A003""","""Stable""",1,2,0,2,1,2,10.0,"""A""","""builtin-attribute-shadowing"""


In [13]:
# the ASYNC rules feel kind of out of place to me as defaults,
# not all projects are going to write any async code. I think
# we could consider revising the Applicability rubric to something like:
# - 2) widely used stdlib types
# - 1) "Niche" stdlib types or very widely used third-party
# - 0) Niche third-party
df.filter(pl.col("linter") == "ASYNC")

rule,type,accuracy,severity,fixability,applicability,configuration,conflicts,total,linter,name
str,str,i64,i64,i64,i64,i64,i64,f64,str,str
"""ASYNC100""","""Stable""",2,2,0,2,2,2,12.0,"""ASYNC""","""cancel-scope-no-checkpoint"""
"""ASYNC220""","""Stable""",2,2,0,2,2,2,12.0,"""ASYNC""","""create-subprocess-in-async-function"""
"""ASYNC221""","""Stable""",2,2,0,2,2,2,12.0,"""ASYNC""","""run-process-in-async-function"""
"""ASYNC222""","""Stable""",2,2,0,2,2,2,12.0,"""ASYNC""","""wait-for-process-in-async-function"""
"""ASYNC251""","""Stable""",2,2,0,2,2,2,12.0,"""ASYNC""","""blocking-sleep-in-async-function"""


In [14]:
# Similarly, the EXE rules probably shouldn't be default
# because they don't even do anything on Windows, and many
# projects may not have any executable Python scripts.
df.filter(pl.col("linter") == "EXE")

rule,type,accuracy,severity,fixability,applicability,configuration,conflicts,total,linter,name
str,str,i64,i64,i64,i64,i64,i64,f64,str,str
"""EXE001""","""Stable""",2,2,0,2,2,2,12.0,"""EXE""","""shebang-not-executable"""
"""EXE002""","""Stable""",2,2,0,2,2,2,12.0,"""EXE""","""shebang-missing-executable-file"""
"""EXE003""","""Stable""",2,2,0,2,2,2,12.0,"""EXE""","""shebang-missing-python"""
"""EXE004""","""Stable""",2,2,2,2,2,2,13.0,"""EXE""","""shebang-leading-whitespace"""
"""EXE005""","""Stable""",2,2,0,2,2,2,12.0,"""EXE""","""shebang-not-first-line"""


## Comparison with Dylan's rules

In [15]:
# fmt: off
prefixes = [
    "YTT",		"SIM103",	"PLE01",	"E7",		"PLE0116",
    "B",		"D419",		"PLE0604",	"PLW0406",	"F701",
    "A",		"F541",		"PLE1310",	"F4",		"F702",
    "C4",		"F601",		"PLE25",	"ARG",		"F703",
    "UP",		"F602",		"PLR5501",	"F8",		"F704",
    "ISC004",	"F631",		"PLW0120",	"PLE0117",	"F705",
    "PIE794",	"F632",		"PLW0129",	"PLE0118",	"F706",
    "PIE796",	"PLC2801",	"PLW0245",	"PLE02",	"F707",
    "PYI",		"PLC1901",	"RUF047",	"PLE4703",	"F722",
]
# fmt: on

dylan_rules = []
for prefix in prefixes:
    if prefix in rules:
        dylan_rules.append(prefix)
    else:
        pat = re.compile(rf"{prefix}\d+")
        expanded_prefix = [rule for rule in rules if pat.match(rule)]
        if len(expanded_prefix) == 0:
            print(f"skipping unknown prefix {prefix}")
        dylan_rules.extend(expanded_prefix)

dylan_rules = set(dylan_rules)
len(dylan_rules)

skipping unknown prefix F703
skipping unknown prefix F705


251

### Rules we both included

In [16]:
df.filter(pl.col("rule").is_in(dylan_rules))

rule,type,accuracy,severity,fixability,applicability,configuration,conflicts,total,linter,name
str,str,i64,i64,i64,i64,i64,i64,f64,str,str
"""YTT101""","""Stable""",2,2,0,2,2,2,12.0,"""YTT""","""sys-version-slice3"""
"""YTT102""","""Stable""",2,2,0,2,2,2,12.0,"""YTT""","""sys-version2"""
"""YTT103""","""Stable""",2,2,0,2,2,2,12.0,"""YTT""","""sys-version-cmp-str3"""
"""YTT201""","""Stable""",2,2,0,2,2,2,12.0,"""YTT""","""sys-version-info0-eq3"""
"""YTT203""","""Stable""",2,2,0,2,2,2,12.0,"""YTT""","""sys-version-info1-cmp-int"""
"""YTT204""","""Stable""",2,2,0,2,2,2,12.0,"""YTT""","""sys-version-info-minor-cmp-int"""
"""YTT301""","""Stable""",2,2,0,2,2,2,12.0,"""YTT""","""sys-version0"""
"""YTT302""","""Stable""",2,2,0,2,2,2,12.0,"""YTT""","""sys-version-cmp-str10"""
"""YTT303""","""Stable""",2,2,0,2,2,2,12.0,"""YTT""","""sys-version-slice1"""


### Rules I included but Dylan didn't

In [17]:
df.filter(~pl.col("rule").is_in(dylan_rules))

rule,type,accuracy,severity,fixability,applicability,configuration,conflicts,total,linter,name
str,str,i64,i64,i64,i64,i64,i64,f64,str,str
"""ASYNC100""","""Stable""",2,2,0,2,2,2,12.0,"""ASYNC""","""cancel-scope-no-checkpoint"""
"""ASYNC220""","""Stable""",2,2,0,2,2,2,12.0,"""ASYNC""","""create-subprocess-in-async-function"""
"""ASYNC221""","""Stable""",2,2,0,2,2,2,12.0,"""ASYNC""","""run-process-in-async-function"""
"""ASYNC222""","""Stable""",2,2,0,2,2,2,12.0,"""ASYNC""","""wait-for-process-in-async-function"""
"""ASYNC251""","""Stable""",2,2,0,2,2,2,12.0,"""ASYNC""","""blocking-sleep-in-async-function"""
"""T100""","""Stable""",2,2,0,2,2,2,12.0,"""T10""","""debugger"""
"""EXE001""","""Stable""",2,2,0,2,2,2,12.0,"""EXE""","""shebang-not-executable"""
"""EXE002""","""Stable""",2,2,0,2,2,2,12.0,"""EXE""","""shebang-missing-executable-file"""
"""EXE003""","""Stable""",2,2,0,2,2,2,12.0,"""EXE""","""shebang-missing-python"""


### Rules Dylan included but I didn't

For a bit of stream-of-consciousness commentary here:

- I'm happy to include YTT202, it just has low applicability because it's an old third-party library
- For most of the B rules with low Accuracy, I think I may have been overly harsh by focusing on open issues
  - I don't think open bug reports are really inherent issues and these could be good in general
- The B rules with low Severity seem accurate to me, but these may be unidiomatic enough to warrant defaults
- The C4 rules, similarly, deal with idioms but probably highly preferable ones
- I gave PYI low Applicability because most projects aren't stubs and many of them are stylistic
- SIM103 was also low Severity but another style I like
- E701-3 I filtered out for redundancy with the formatter, and the other E rules are stylistic
- I gave D419 a very low severity, but it actually does seem like a potential mistake, I think I'll steal that one
- F541 also seems like a potential error on second thought, as does PLE0237
- PLR5501 is also an idiom but another good one
- I filtered out many of the `UP` rules for being stylistic and depending on `target-version`, but like I note below,
  I like these rules and would be happy to include them

In [18]:
full.filter(pl.col("rule").is_in(dylan_rules)).join(df, on="rule", how="anti").filter(
    pl.col("type") == "Stable"
)

rule,type,accuracy,severity,fixability,applicability,configuration,conflicts,total,linter,name
str,str,i64,i64,i64,i64,i64,i64,f64,str,str
"""YTT202""","""Stable""",2,2,0,0,2,2,10.0,"""YTT""","""six-py3"""
"""B006""","""Stable""",1,2,2,2,1,2,11.0,"""B""","""mutable-argument-default"""
"""B007""","""Stable""",0,2,2,2,2,2,11.0,"""B""","""unused-loop-control-variable"""
"""B008""","""Stable""",1,2,0,2,1,2,10.0,"""B""","""function-call-in-default-argument"""
"""B009""","""Stable""",2,1,2,2,2,2,11.0,"""B""","""get-attr-with-constant"""
"""B010""","""Stable""",2,1,2,2,2,2,11.0,"""B""","""set-attr-with-constant"""
"""B011""","""Stable""",2,1,2,2,2,2,11.0,"""B""","""assert-false"""
"""B013""","""Stable""",2,1,2,2,2,2,11.0,"""B""","""redundant-tuple-in-exception-handler"""
"""B014""","""Stable""",2,1,2,2,2,2,11.0,"""B""","""duplicate-handler-exception"""


## Proposed defaults

In [19]:
filter_out_linters = [
    # See above, not all projects use async
    "ASYNC",
    # Similarly, not all projects have executable files, and
    # these rules don't do anything on some systems (Windows)
    "EXE",
    # These are related to text translations, which also doesn't
    # seem relevant for most projects
    "INT",
]

filter_out_rules = [
    # ambiguous-unicode-character rules, these seem too noisy
    # for non-english text and possibly some mathematical code too
    "RUF001",
    "RUF002",
    "RUF003",
]

additional_rules = [
    # I was overly harsh on false negative reports, and I think
    # it makes sense to enable all of the A rules
    "A001",
    "A003",
    # An empty docstring does seem like a likely mistake
    "D419",
    # As does an f-string without placeholders
    "F541",
    # And invalid syntax in an annotation
    "F722",
    # Known false negatives, but a runtime error to violate
    "PLE0237",
]

# I haven't filtered these out yet, just flagging them for additional input
questionable_rules = [
    # This is pretty niche actually, not sure it's worth including
    "FURB163",
    # These have known false positives with the `loguru` library because
    # it uses `{}` for formatting instead of `%`. I'm not sure exactly how
    # common loguru is, but we get reports about it pretty often.
    "PLE1205",
    "PLE1206",
    # These two feel _kind of_ opinionated to me, I definitely wouldn't oppose
    # dropping them.
    "PLW1509",
    "PLW1510",
]

# I also haven't filtered these out yet, but they seem like they could be
# caught by a type checker (and probably should have Conflicts = 1)
typing_related = [
    "PLE1507",
    "RUF016",
    # ty actually doesn't flag this for some reason:
    # https://play.ty.dev/5d89172d-0df3-43de-a91a-1aa9d3d23516
    #
    # but I think it should trigger:
    # https://docs.astral.sh/ty/reference/rules/#positional-only-parameter-as-kwarg
    "RUF026",
]

# I think these are already in the list, but make sure to include them because
# they correspond to syntax errors. See the appendix for some additional notes.
syntax_errors = [
    "F404",
    "F407",
    "F622",
    "F701",
    "F702",
    "F704",
    "F706",
    "F707",
    "PLE0115",
    "PLE0116",
    "PLE0117",
    "PLE0118",
    "PLE1142",
    "PLE1700",
]

filtered = df.filter(
    ~pl.col("linter").is_in(filter_out_linters),
    ~pl.col("rule").is_in(filter_out_rules),
)

proposed = (
    pl
    .concat(
        [
            filtered,
            full.filter(pl.col("rule").is_in(missing_current_defaults)),
            full.filter(pl.col("rule").is_in(additional_rules)),
            full.filter(pl.col("rule").is_in(syntax_errors)),
        ],
        how="vertical",
    )
    .unique()
    .sort("rule")
)

proposed

rule,type,accuracy,severity,fixability,applicability,configuration,conflicts,total,linter,name
str,str,i64,i64,i64,i64,i64,i64,f64,str,str
"""A001""","""Stable""",1,2,0,2,1,2,10.0,"""A""","""builtin-variable-shadowing"""
"""A002""","""Stable""",2,2,0,2,1,2,11.0,"""A""","""builtin-argument-shadowing"""
"""A003""","""Stable""",1,2,0,2,1,2,10.0,"""A""","""builtin-attribute-shadowing"""
"""A004""","""Stable""",2,2,0,2,1,2,11.0,"""A""","""builtin-import-shadowing"""
"""A005""","""Stable""",2,2,0,2,1,2,11.0,"""A""","""stdlib-module-shadowing"""
"""A006""","""Stable""",2,2,0,2,1,2,11.0,"""A""","""builtin-lambda-argument-shadowing"""
"""ARG001""","""Stable""",2,2,0,2,1,2,11.0,"""ARG""","""unused-function-argument"""
"""ARG002""","""Stable""",2,2,0,2,1,2,11.0,"""ARG""","""unused-method-argument"""
"""ARG003""","""Stable""",2,2,0,2,1,2,11.0,"""ARG""","""unused-class-method-argument"""


In [20]:
def to_url(rule: str) -> str:
    return f"[{rule}](https://docs.astral.sh/ruff/rules/{rule})"


with (
    pl.Config(
        tbl_formatting="MARKDOWN",
        tbl_hide_column_data_types=True,
        tbl_hide_dataframe_shape=True,
        tbl_width_chars=-1,
    ),
    open("rules.md", "w") as f,
):
    print(
        proposed.select("rule", pl.col("name").map_elements(to_url).alias("name")),
        file=f,
    )

## Appendix

In [21]:
def by_linter(linter):
    return pl.col("linter").is_in((linter,)) & (pl.col("type") == "Stable")

### `FURB163`

I was curious why only this FURB rule appeared in my list of rule candidates.
I guess it's because it had the only severity of 2, which I assigned based on
the fact that manually providing the base can produce less accurate results,
according to our docs.

The other FURB rules also scored highly but were filtered out for being less severe.

This feels fairly niche and unlikely to cause errors that are too substantial, so
I definitely wouldn't be opposed to removing this from the set.

In [22]:
full.filter(by_linter("FURB"))

rule,type,accuracy,severity,fixability,applicability,configuration,conflicts,total,linter,name
str,str,i64,i64,i64,i64,i64,i64,f64,str,str
"""FURB105""","""Stable""",2,1,2,2,2,2,11.0,"""FURB""","""print-empty-string"""
"""FURB116""","""Stable""",2,1,1,2,2,2,10.5,"""FURB""","""f-string-number-format"""
"""FURB122""","""Stable""",2,1,2,2,2,2,11.0,"""FURB""","""for-loop-writes"""
"""FURB129""","""Stable""",2,1,2,2,2,2,11.0,"""FURB""","""readlines-in-for"""
"""FURB132""","""Stable""",2,1,2,2,2,2,11.0,"""FURB""","""check-and-remove-from-set"""
"""FURB136""","""Stable""",2,1,2,2,2,2,11.0,"""FURB""","""if-expr-min-max"""
"""FURB157""","""Stable""",2,1,2,2,2,2,11.0,"""FURB""","""verbose-decimal-constructor"""
"""FURB161""","""Stable""",2,1,2,2,1,2,10.0,"""FURB""","""bit-count"""
"""FURB162""","""Stable""",2,1,2,2,2,2,11.0,"""FURB""","""fromisoformat-replace-z"""


### LOG001

Like `FURB163`, this is the only rule from its linter included, and I think the reason is the same.
Our docs excerpt from the Python docs:

> Note that Loggers should _NEVER_ be instantiated directly

I considered the other LOG rules less severe.

In [23]:
full.filter(by_linter("LOG"))

rule,type,accuracy,severity,fixability,applicability,configuration,conflicts,total,linter,name
str,str,i64,i64,i64,i64,i64,i64,f64,str,str
"""LOG001""","""Stable""",2,2,2,2,2,2,13.0,"""LOG""","""direct-logger-instantiation"""
"""LOG002""","""Stable""",2,1,2,2,2,2,11.0,"""LOG""","""invalid-get-logger-argument"""
"""LOG007""","""Stable""",2,1,0,2,1,2,9.0,"""LOG""","""exception-without-exc-info"""
"""LOG009""","""Stable""",2,1,2,2,2,2,11.0,"""LOG""","""undocumented-warn"""
"""LOG014""","""Stable""",0,1,2,2,1,2,8.0,"""LOG""","""exc-info-outside-except-handler"""
"""LOG015""","""Stable""",2,1,0,2,2,2,10.0,"""LOG""","""root-logger-call"""


### Syntax Errors

As Dylan noted, we should just remove these rules and make them syntax errors at some point.
Making them default may be a good step in that direction? 

`E999` is literally a syntax error obviously, but we removed it. `F722` is also literally a
syntax error, but I consider it a bit different since it's not evaluated by CPython at runtime.

In [24]:
def red(text):
    return f"\x1b[31;1m{text}\x1b[0m"


syntax_error = re.compile(r"syntax.*error", flags=re.IGNORECASE)
non_errors = ["B017", "PIE790", "E999", "F722", "FURB116"]
true_errors = [
    "F404",
    "F407",
    "F622",
    "F701",
    "F702",
    "F704",
    "F706",
    "F707",
    "PLE0115",
    "PLE0116",
    "PLE0117",
    "PLE0118",
    "PLE1142",
    "PLE1700",
]
for rule in (
    rule
    for rule in rule_data
    if syntax_error.search(rule["explanation"])
    and rule["code"] not in non_errors
    and rule["code"] not in true_errors
):
    print(rule["name"], "-", rule["code"])
    print(syntax_error.sub(red("SyntaxError"), rule["explanation"]))
    print("=" * 80)

### UP

It looks like I filtered the UP rules pretty aggressively, I think because
many of them either won't work or won't work properly unless the `target-version` 
is configured. I personally quite like these rules and would be happy enabling ~all 
of them by default, but I was conservative with the scoring here.